# LangChain Multi-Agent

This notebook demonstrates how to roughly use LangChain to create a multi-agent workflow.

## Setup and Installation

First, let's install the required packages and verify the installation:

In [ ]:
# Install required packages
!pip install langchain openai python-dotenv langchain_community

# Verify installations
import importlib

def check_package(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False

packages = {
    'langchain': 'LangChain',
    'openai': 'OpenAI',
    'dotenv': 'python-dotenv',
    'langchain_community': 'LangChain Community'
}

all_installed = True
for package, display_name in packages.items():
    installed = check_package(package)
    print(f"{display_name}: {'✅ Installed' if installed else '❌ Not installed'}")
    all_installed = all_installed and installed

if all_installed:
    print("\n✅ All required packages are installed!")
else:
    print("\n⚠️ Some packages are missing. Run the installation command again.")

## Environment Setup

Load environment variables from the `.env` file. <br>
N.b. it will look through the entire project for a valid `.env` file.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Set environment variables for compatibility with libraries that expect them
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY or ""

# Verify API keys are set
if not OPENAI_API_KEY:
    print("⚠️ Warning: OPENAI_API_KEY is not set in .env file")
else:
    print("✅ API keys are set")

## Purpose
As your functionalities get longer and gain complexity, you will likely need to create multiple agents and as such use an agentic workflow.

We shall hence create multiple agents for this example, including:

- Writer Agent – Creates content (e.g., blog post).
- Editor Agent – Reviews and improves clarity, grammar, and tone.
- Summarizer Agent – Turns the final output into bullet points or TL;DR.

Each of these agent will be provided with their own example tools.

## Define Tools

In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

# --- Tools for Writer ---
writer_tools = [
    Tool(
        name="Write Blog",
        func=lambda topic: llm.invoke(f"Write a detailed and engaging blog post about: {topic}"),
        description="Writes a blog post on a given topic."
    )
]

# --- Tools for Editor ---
editor_tools = [
    Tool(
        name="Edit Text",
        func=lambda text: llm.invoke(f"Improve this text for grammar, clarity, and tone:\n\n{text}"),
        description="Improves grammar and clarity of a text."
    )
]

# --- Tools for Summarizer ---
summarizer_tools = [
    Tool(
        name="Summarize Text",
        func=lambda text: llm.invoke(f"Summarize this in 5 bullet points:\n\n{text}"),
        description="Summarizes text into bullet points."
    )
]


## Create each Agents

In [ ]:
# Writer Agent
writer_agent = initialize_agent(
    tools=writer_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Editor Agent
editor_agent = initialize_agent(
    tools=editor_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Summarizer Agent (optional)
summarizer_agent = initialize_agent(
    tools=summarizer_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


## Create a rudimentary Workflow

In [ ]:
# Step 1: Writer creates the content
topic = "How to make an apple pie"
raw_draft = writer_agent.run(f"Write a blog post about: {topic}")

# Step 2: Editor refines the content
edited_draft = editor_agent.run(f"Please improve the following blog post:\n\n{raw_draft}")

# Step 3: Summarizer makes a TL;DR (optional)
summary = summarizer_agent.run(f"Summarize this post:\n\n{edited_draft}")

# Final Output
print("📝 Raw Draft:\n", raw_draft)
print("✍️ Final Post:\n", edited_draft)
print("\n🧾 TL;DR:\n", summary)


## What's next ?
While this solution to use Multi-Agent is possible, we strongly encourages you to use LangChainGraph to create Workflows. This example being simply a text of the possibilities provided by LangChain.